# Here we analayse the Results of ChatGPT for the input with text and annotations

In [ ]:
from support_functions.load_json_chatgpt_results import load_chat_gpt_results

## Load ChatGPT results

In [ ]:
directories_to_ignore:dict = ["annoations"]
chat_gpt_results: dict[str, dict[str, dict]] = load_chat_gpt_results(directories_to_ignore)
chat_gpt_result_keys: list[str] = list(chat_gpt_results.keys())
chat_gpt_result_json_keys = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in chat_gpt_results.items()
}

## Analyse the data sets